<a href="https://colab.research.google.com/github/Sudol67/MachineLearningEyeVeins/blob/main/DnoOkaV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from PIL import Image

Importowane biblioteki:
os -
cv2 - wczytywanie i edycja zdjęć
matplotlib - wyswietlanie

In [28]:
folder_path_before = '/content/Before'
folder_path_manual = '/content/manual'

original_images = {}
manual_images = {}

for img_file in os.listdir(folder_path_before):
    if img_file.endswith(".jpg"):
        base_name = os.path.splitext(img_file)[0]  # Usuwanie rozszerzenia pliku
        image_path = os.path.join(folder_path_before, img_file)
        # Wczytanie obrazu w skali szarości
        image = cv2.imread(image_path)
        if image is not None:
            original_images[base_name] = image
        else:
            print(f"Nie udało się wczytać obrazu {img_file} z folderu Before.")

# Wczytywanie obrazów .tif z folderu 'manual'
for img_file in os.listdir(folder_path_manual):
    if img_file.endswith(".tif"):
        base_name = os.path.splitext(img_file)[0]
        image_path = os.path.join(folder_path_manual, img_file)
        # Używamy PIL do wczytania, ponieważ cv2 może mieć problemy z niektórymi plikami .tif
        pil_image = Image.open(image_path)
        image_array = np.array(pil_image)
        if image is not None:
            manual_images[base_name] = image_array
        else:
            print(f"Nie udało się wczytać obrazu {img_file} z folderu manual.")

Tworzenie listy obrazów z folderu

In [29]:
processed_images_before = {}

for img_file, image in original_images.items():
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    equalized_image = cv2.equalizeHist(gray_image)
    sharpening_kernel = np.array([[-1, -1, -1],
                                  [-1,  9, -1],
                                  [-1, -1, -1]])
    sharpened_image = cv2.filter2D(equalized_image, -1, sharpening_kernel)
    processed_images_before[img_file] = sharpened_image

Wstępna obróbka wczytanych obrazów: zamiana kolorów na skalę szarości, wyrównanie histogramu dla poprawy kontrastu, wyostrzenie

In [30]:
def display_image(image_name):
    if image_name in processed_images_before:
        plt.imshow(processed_images_before[image_name], cmap='gray')
        plt.title(f'Wyświetlany Obraz: {image_name}')
        plt.show()
    else:
        print("Nie ma takiego obrazu.")

display_image('01_h.jpg')

Nie ma takiego obrazu.


Tworzenie macierzy cech pikseli czyli tworzenie zbioru uczącego

In [31]:
def calculate_features(image, x, y):
    pixel_value = image[x, y]
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)[x, y]
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)[x, y] # gradient
    features = [pixel_value, sobelx, sobely]
    return features

X = []  # lista cech pikseli dla wszystkich obrazów
labels = []  # naczynie czy nie

for img_file, processed_image in processed_images_before.items():
    manual_image = manual_images[img_file]

    for x in range(processed_image.shape[0]):
        for y in range(processed_image.shape[1]):
            features = calculate_features(processed_image, x, y)
            X.append(features)

            is_vein = manual_image[x, y] > 127
            labels.append(int(is_vein))

X = np.array(X)
y = np.array(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

KeyboardInterrupt: 